# Trials:

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense, Concatenate, Flatten
from tensorflow.keras.models import Model

NameError: name '_C' is not defined

In [ ]:
# Load your dataset (replace 'your_dataset.csv' with your actual dataset)
# Assuming your dataset has columns: 'Temperature', 'Vibration_X', 'Vibration_Y', 'Vibration_Z', and 'Target'
# dataset = pd.read_csv("data_1.csv")
dataset = pd.read_csv("/content/drive/MyDrive/V Sem7/End-Sem Project/Data/Material 1/data_1.csv")

# Extract features and target variable
X = dataset[["Temperature", "Vibration_X", "Vibration_Y", "Vibration_Z"]].values

#  Confusion here is with the 'target' value... letss see
# y = dataset["Target"].values
y = dataset["Temperature"].values


print(X)
print('------------------------')
print(y)

[[   26.84    -2.68    23.09 -1014.33]
 [   30.19   -23.72    23.3  -1011.56]
 [   29.48     1.24    19.01 -1012.23]
 ...
 [   28.08    -8.38    31.9  -1024.12]
 [   27.01    -1.48    19.46 -1019.94]
 [   25.69   -23.31    26.69 -1028.95]]
------------------------
[26.84 30.19 29.48 ... 28.08 27.01 25.69]


In [ ]:
# Normalize the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_3d = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))


In [ ]:
X_train_3d

array([[[0.78473091, 0.06742323, 0.50016683, 0.04126984]],

       [[0.17897372, 0.06909212, 0.75008342, 0.2037037 ]],

       [[0.79474343, 0.6975968 , 0.00734067, 0.31693122]],

       ...,

       [[0.89111389, 0.3411215 , 0.95762429, 0.12010582]],

       [[0.54443054, 0.34445928, 0.67600934, 0.61851852]],

       [[0.61451815, 0.12917223, 0.40707374, 0.02433862]]])

--------------------------------------------------------------------

# Trying logic:

In [ ]:
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, Concatenate, Attention

# Define the input layer
input_layer = Input(shape=(X.shape[1], 1))

# Create Stacked LSTM models with Transformer-like layers
model_vibration = Sequential()
model_temperature = Sequential()

# Add Stacked LSTM layers
model_vibration.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(X.shape[1], 1)))
model_vibration.add(LSTM(units=50, activation='relu'))
model_vibration.add(Dense(units=X.shape[1]))  # Output layer for vibration

model_temperature.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(X.shape[1], 1)))
model_temperature.add(LSTM(units=50, activation='relu'))
model_temperature.add(Dense(units=1))  # Output layer for temperature

# Apply Attention mechanism
attention_vibration = Attention()([model_vibration(input_layer), input_layer])
attended_vibration = Concatenate(axis=-1)([input_layer, attention_vibration])
output_vibration = Dense(units=X.shape[1])(attended_vibration)

attention_temperature = Attention()([model_temperature(input_layer), input_layer])
attended_temperature = Concatenate(axis=-1)([input_layer, attention_temperature])
output_temperature = Dense(units=1)(attended_temperature)

# Create the final model
model_vibration = Model(inputs=input_layer, outputs=output_vibration)
model_temperature = Model(inputs=input_layer, outputs=output_temperature)

# Compile the models
model_vibration.compile(optimizer='adam', loss='mean_squared_error')
model_temperature.compile(optimizer='adam', loss='mean_squared_error')

# Train the models
model_vibration.fit(X_stacked_lstm_transformer, y_vibration, epochs=100, batch_size=32)
model_temperature.fit(X_stacked_lstm_transformer, y_temperature, epochs=100, batch_size=32)

# Predict the 11th day values
X_11th_day_stacked_lstm_transformer = np.array(X.iloc[[-1]]).reshape(1, X.shape[1], 1)
vibration_11th_day_prediction_stacked_lstm_transformer = model_vibration.predict(X_11th_day_stacked_lstm_transformer)
temperature_11th_day_prediction_stacked_lstm_transformer = model_temperature.predict(X_11th_day_stacked_lstm_transformer)

# Actual values for the 11th day
actual_vibration_11th_day = np.array([[-10, 18, -1018]])  # Replace with the actual values
actual_temperature_11th_day = 30.0  # Replace with the actual value

# Calculate errors
vibration_error_stacked_lstm_transformer = np.abs(vibration_11th_day_prediction_stacked_lstm_transformer[0] - actual_vibration_11th_day)
temperature_error_stacked_lstm_transformer = np.abs(temperature_11th_day_prediction_stacked_lstm_transformer[0][0] - actual_temperature_11th_day)

# Display results
print("Stacked LSTM with Transformer-like layer - Actual Vibration on the 11th day:", actual_vibration_11th_day)
print("Stacked LSTM with Transformer-like layer - Predicted Vibration on the 11th day:", vibration_11th_day_prediction_stacked_lstm_transformer[0])
print("Stacked LSTM with Transformer-like layer - Vibration Error for the 11th day:", vibration_error_stacked_lstm_transformer)

print("\nStacked LSTM with Transformer-like layer - Actual Temperature on the 11th day:", actual_temperature_11th_day)
print("Stacked LSTM with Transformer-like layer - Predicted Temperature on the 11th day:", temperature_11th_day_prediction_stacked_lstm_transformer[0][0])
print("Stacked LSTM with Transformer-like layer - Temperature Error for the 11th day:", temperature_error_stacked_lstm_transformer)
